In [1]:
import requests 
import os
import re
from collections import defaultdict
from copy import deepcopy
import pickle
import pandas as pd
import numpy as np
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, StaleElementReferenceException

In [3]:
from Crawl_Naver_Shopping import *
import pickle

In [5]:
crawl = Crawl_Naver_Shopping(list_type = "가격비교")

In [ ]:
crawl.get_product_infos_from_query(queries = ["27ART10AKPL", "KU50UA7050FXKR", "W20WAN"])

In [9]:
with open("category_ids_220721.pkl", "rb") as f:
    unq_category_ids = pickle.load(f)

with open("category_levels_220721.pkl", "rb") as f:
    unq_category_levels = pickle.load(f)

In [11]:
unq_category_levels

[['디지털/가전', '노트북'],
 ['디지털/가전', '노트북액세서리'],
 ['디지털/가전', '노트북액세서리', '노트북가방/케이스'],
 ['디지털/가전', '노트북액세서리', '노트북받침대/쿨러'],
 ['디지털/가전', '노트북액세서리', '노트북보호필름'],
 ['디지털/가전', '노트북액세서리', '노트북보호필름', '액정보호필름'],
 ['디지털/가전', '노트북액세서리', '노트북보호필름', '전신보호필름'],
 ['디지털/가전', '노트북액세서리', '노트북보안기'],
 ['디지털/가전', '노트북액세서리', '노트북어댑터'],
 ['디지털/가전', '노트북액세서리', '노트북용배터리'],
 ['디지털/가전', '노트북액세서리', '노트북키스킨'],
 ['디지털/가전', '노트북액세서리', '노트북도난방지'],
 ['디지털/가전', '노트북액세서리', '기타노트북액세서리'],
 ['디지털/가전', '휴대폰액세서리'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스', '가죽케이스'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스', '실리콘케이스'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스', '플라스틱케이스'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스', '메탈케이스'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스', '기타케이스'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰케이스', '암밴드'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰보호필름'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰보호필름', '액정보호필름'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰보호필름', '전신보호필름'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰보호필름', '휴대폰스킨'],
 ['디지털/가전', '휴대폰액세서리', '휴대폰보호필름', '홈버튼스티커'],
 ['디지털/가전', '휴대폰액

In [4]:
# 스탠드 제품 코드 추가
stand_ids = ["50003330", "50003331"]
stand_levels = [["가구/인테리어", "조명/스탠드", "스탠드", "단스탠드"], 
 ["가구/인테리어", "조명/스탠드", "스탠드", "장스탠드"]]

unq_category_ids.extend(stand_ids)
unq_category_levels.extend(stand_levels)

In [5]:
category_list = ["LEDTV", "OLEDTV", "LCDTV", "QLEDTV", "DLP", "LCD", "LCoS/기타", 
"퍼스널뷰어", "블루레이플레이어", "사운드바시스템", "단일형홈시어터", 
"조합형홈시어터", "일체형오디오", "스피커단품", "스피커세트", "블루투스스피커", 
"블루투스이어폰/이어셋", "블루투스헤드폰/헤드셋", "이어폰", "헤드폰", 
"일반세탁기", "드럼세탁기", "미니세탁기", "세탁기+건조기", "의류건조기", 
"신발건조기", "탈수기", "유선청소기", "무선청소기", "스팀청소기", 
"로봇청소기", "물걸레청소기", "연수기", "단스탠드", "장스탠드", "의류관리기", 
"스팀다리미", "보풀제거기", "전동칫솔", "구강세정기", "칫솔살균기", 
"전신건조기", "드라이어", "고데기", "매직기", "에어브러시", "남성용면도기", 
"여성용면도기", "두피케어기기", "제모기", "양문형냉장고", "일반형냉장고", 
"스탠드형", "냉동고", "와인냉장고", "화장품냉장고", "쌀냉장고", "제빙기", 
"일반가스레인지", "일반밥솥", "전자레인지", "전기오븐", "복합형오븐", "인덕션",
"핫플레이트", "하이라이트", "하이브리드", "식기세척기", "식기건조기", "무선포트", 
"유선포트", "냉정수기", "냉온정수기", "에스프레소머신", "캡슐/POD머신", 
"팝업토스터기", "오븐토스터기", "음식물처리기", "제빵기", "거품/반죽기", 
"전기그릴", "생선그릴", "쥬서기/녹즙기", "핸드블렌더", "슬로우쿠커", "전기냄비", 
"전기찜기", "튀김기", "전기팬", "샌드위치제조기", "와플제조기", "탄산수제조기", 
"에어프라이어", "믹서기", "전기담요", "전기방석", "전기요", "황토매트", "카페트매트", 
"옥매트", "기타전기매트", "전기장판", "전기온풍기", "가스온풍기", "석유온풍기", "돈풍기", 
"가스보일러", "석유보일러", "전기보일러", "전기온수기", "가스온수기", "라디에이터", 
"컨벡터", "스탠드형에어컨", "벽걸이형에어컨", "멀티형에어컨", "창문형에어컨", "일반용제습기", 
"냉온풍기", "자연식가습기", "가열식가습기", "초음파식가습기", "복합식가습기", "공기청정기", 
"에어워셔", "일반용냉풍기", "스탠드형선풍기", "탁상형선풍기", "휴대용선풍기", "천장형선풍기", 
"전기히터", "석유히터", "가스히터", "연탄/화목난로", "안마의자", "안마기"]
print(f"length of sample category list : {len(category_list)}")

length of sample category list : 136


In [6]:
sample_cat_ids = [unq_category_ids[i] for i, x in enumerate(unq_category_levels) if x[-1] in category_list]
print(f"length of sample cat ids : {len(sample_cat_ids)}")
sample_cat_levels = [x for i, x in enumerate(unq_category_levels) if x[-1] in category_list]
print(f"length of sample cat levels : {len(sample_cat_levels)}")

length of sample cat ids : 138
length of sample cat levels : 138


In [7]:
sample_cat_ids.extend(['50000151', '50000089', '50000088', '50000152'])
sample_cat_levels.extend(["['디지털/가전', '노트북']'", "['디지털/가전', 'PC']", "['디지털/가전', '게임기/타이틀']", "['디지털/가전', '태블릿PC']"])
print(f"length of sample cat ids : {len(sample_cat_ids)}")
print(f"length of sample cat levels : {len(sample_cat_levels)}")

length of sample cat ids : 142
length of sample cat levels : 142


In [ ]:
crawl = Crawl_Naver_Shopping(list_type="가격비교")
crawl_result = crawl.get_product_infos_in_category(cat_ids = unq_category_ids, cat_levels = unq_category_levels)

In [7]:
pd.DataFrame(crawl_result).to_excel("네이버_쇼핑_크롤링_220426.xlsx", index =False)

In [6]:
light_stand_df = pd.read_excel("조명_스탠드_수집_220401.xlsx")
digital_df  = pd.read_excel("가전_디지털_220401.xlsx")

In [7]:
product_data = pd.concat([digital_df.drop_duplicates("id"), light_stand_df.drop_duplicates("id")], axis = 0)

In [10]:
# pd_crawl_result = pd.DataFrame(crawl_result)
product_data = product_data[['searchCategoryId', 'searchCategoryName', 'id', 'productName', 'scoreInfo', 'category1Id', 'category2Id',
                             'category3Id', 'category4Id', 'category1Name', 'category2Name',
                             'category3Name', 'category4Name', 'maker', 'makerNo', 'brand',
                             'brandNo', 'price', 'priceUnit', 'lnchYm', 'productAttr', 'imageName', 'crUrl', 'imageUrl']]

In [ ]:
product_data.to_excel("네이버_쇼핑_크롤링_220404.xlsx", index = False)